In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *
from sympy import Matrix, symbols, solve_linear_system
from sympy.solvers.inequalities import *
from sympy.polys import Poly
from sympy.abc import x
from sympy.solvers.solveset import linsolve
import time
from joblib import Parallel, delayed
x,y,z_1,z_2,z_3 = symbols('x y z_1 z_2 z_3')
x,y,z1,z2,z3 ,b= symbols("x,y,z1,z2,z3,b")
p_112, p_102, p_002, p_012 = symbols('p_112 p_102 p_002 p_012')
p_000, p_001, p_100, p_101, p_200, p_201, p_010, p_011, p_020, p_021, p_110, p_111, p_210, p_211, p_120, p_121, p_220, p_221 = symbols('p_000 p_001 p_100 p_101 p_200 p_201 p_010 p_011 p_020 p_021 p_110 p_111 p_210 p_211 p_120 p_121 p_220 p_221')

In [2]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False

In [3]:
#generates random rank 3 tensors
def rank_tree(size = 100):
    return (low_tensor(size) + low_tensor(size) + low_tensor(size)) 

# random tensor of some dim
def rand_tensor(size = 100000):
    return tl.tensor(np.random.randint(1, size, size=(3,2,2)))*1.0

# generates rank 1 tensors
def low_tensor(size = 100):
    a = np.random.randint(1, size, size=3) 
    b = np.random.randint(1, size, size=2)
    c = np.random.randint(1, size, size=2)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

# checks if all subtensors have nonneg rank 3
def check(t):
    t1 = tl.tensor([t[0], t[1]])
    t2 = tl.tensor([t[1], t[2]])
    t3 = tl.tensor([t[0], t[2]])
    a1 = det(Matrix(t1[0]))
    a2 = det(Matrix(t1[1]))
    a3 = det(Matrix(t2[0]))
    a4 = det(Matrix(t2[1]))
    b1 = det(Matrix(t1[:,0]))
    b2 = det(Matrix(t1[:,1]))
    b3 = det(Matrix(t2[:,0]))
    b4 = det(Matrix(t2[:,1]))
    c1 = det(Matrix(t1[:,:,0]))
    c2 = det(Matrix(t1[:,:,1]))
    c3 = det(Matrix(t2[:,:,0]))
    c4 = det(Matrix(t2[:,:,1]))

    a5 = det(Matrix(t3[0]))
    a6 = det(Matrix(t3[1]))

    b5 = det(Matrix(t3[:,0]))
    b6 = det(Matrix(t3[:,1]))

    c5 = det(Matrix(t3[:,:,0]))
    c6 = det(Matrix(t3[:,:,1]))
    return sgn([a1, a2,a3,a4,a5,a6]) or sgn([b1,b2,b3,b4,b5,b6]) or sgn([c1,c2,c3,c4,c5,c6])

# determines sign of a list
def sgn(a):
    t = 0
    ab = 0
    for a_i in a:
        t+= abs(a_i)
        ab += a_i
    return t == abs(ab)


# checks if the give 2x2x2 tensor has nonneg rank 3
def check_r3(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    return sgn([a1,a2]) or sgn([b1,b2]) or sgn([c1,c2])

# checks if given 2x2x2 has nonneg rank 2
def check_r2(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    d1 = ineq(a1,a2,b1,b2,c1,c2,ge,ge,ge)
    d2 = ineq(a1,a2,b1,b2,c1,c2,le,le,ge)
    d3 = ineq(a1,a2,b1,b2,c1,c2,le,ge,le)
    d4 = ineq(a1,a2,b1,b2,c1,c2,ge,le,le)
    supermod = d1 or d2 or d3 or d4
    return supermod


# helpers
def ineq(a1,a2,b1,b2,c1,c2,f1,f2,f3):
    t1 = f1(a1,0) and f1(a2,0)
    t2 = f2(b1,0) and f2(b2,0)
    t3 = f3(c1,0) and f3(c2,0)
    return t1 and t2 and t3
    
def ge(a1,a2):
    return a1 >= a2

def le(a1,a2):
    return a1 <= a2

# Checks if 2x2x3 contains a 2x2x2 with nonneg 4
def check_r4(t, upper = 1):
    a1 = not check_r3(tl.tensor([t[0],t[1]]))
    a2 = not check_r3(tl.tensor([t[0],t[2]]))
    a3 = not check_r3(tl.tensor([t[1],t[2]]))
    return (a1 + a2 + a3) >= upper

def r2_sub(t, upper = 1):
    a = check_r2(tl.tensor([t[0],t[1]]))
    b = check_r2(tl.tensor([t[0],t[2]]))
    c = check_r2(tl.tensor([t[2],t[1]]))
    return (bool(a)+bool(b)+bool(c)) >= upper

In [4]:
def mat_inv(t):
    tens = t.copy()
    M_1 = Matrix([Matrix(tens[0][1]).transpose(),Matrix(tens[0][0]).transpose()])
    M_2 = Matrix([Matrix(tens[1][1]).transpose(),Matrix(tens[1][0]).transpose()])
    M_3 = Matrix([Matrix(tens[2][1]).transpose(),Matrix(tens[2][0]).transpose()])
    tens[0] = M_1
    tens[1] = M_2
    tens[2] = M_3
    return tens

def mat_trans(t):
    tens = t.copy()
    tens[0] = tens[0].transpose()
    tens[1] = tens[1].transpose()
    tens[2] = tens[2].transpose()
    return tens

def rotate(t):
    return mat_inv(mat_trans(t))


def proc_tensor(t):
    tens = t.copy()
    M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
    M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
    if abs(M_a.det()) < 10:
        print("warning det too small")
    a = 1
    b = M_b.det() / M_a.det()
    M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
                [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
                [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
    R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
    M_sub = Matrix(M[0:5,0:5])
    #sol = np.array(np.array(M_sub.inv(), dtype = "float") @ np.array(Matrix(R[0:5]), dtype = "float"), dtype = "float")
    sol = la.solve(np.array(M.T@M,dtype = "float"), np.array(M.T@R, dtype = "float"))
    ret = np.append(np.array([b], dtype = "float"),sol)
    return np.all(ret >=0)


def test(tens):
    ret = [0] *11
    if proc_tensor(tens) or proc_tensor(mat_trans(tens)) or proc_tensor(mat_inv(tens)) or proc_tensor(rotate(tens)):
        ret[1] = 1
        ret[0] = 1
    if check_simple(tens):
        ret[2] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 2)
    #ret[5] = r2
    #if check_r4(tens, upper = 1):
    #    ret[6] = 1
    #    if r2_sub(tens, upper = 1):
    #        ret[9] = 1
    #        if ret[0] == 1:
    #            ret[10] = 1
    r4 = check_r4(tens)
    if r4  and r2:
        ret[3] = 1
    if not r4 and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    return ret

def test_r2(tens):
    A = Matrix(tens[0]) * Matrix(tens[1]).inv()
    res1 = Matrix(A.eigenvects()[0][2])
    res2 = Matrix(A.eigenvects()[1][2])

    B = Matrix(tens[0]).transpose() * Matrix(tens[1]).transpose().inv()
    res3 = Matrix(B.eigenvects()[0][2])
    res4 = Matrix(B.eigenvects()[1][2])
    
    T1 = np.kron(res1,res3) * 1.0
    T2 = np.kron(res2,res4) * 1.0
    assert(np.all(T1 >= 0) or np.all(T1 <= 0))
    assert(np.all(T2 >= 0) or np.all(T2 <= 0))
    T1 = Matrix(abs(T1))
    T2 = Matrix(abs(T2))

    a = symbols('a')
    M = Matrix([[T1,T2]])
    A1 = Matrix([a,a,1,1])
    A2 = Matrix([a,1,a,1])
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])

    a1 = solve(M1.det())[0]
    A1 = Matrix([a1,a1,1,1])
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M1_sub = M1[0:3,0:3]
    R = M1[0:3,3]
    d1 = False
    if abs(M1_sub.det()) > 0:
        c1 = np.array(M1_sub.inv() @ R, dtype = "float")
        d1 = np.all(c1 >=0)

    a2 = solve(M2.det())[0]
    A2 = Matrix([a2,1,a2,1])
    M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
    M2_sub = M2[0:3,0:3]
    R = M2[0:3,3]
    d2 = False
    if abs(M2_sub.det()) > 0:
        c2 = np.array(M2_sub.inv() @ R, dtype = "float")
        d2 = np.all(c2 >=0)
    return d1 or d2

def check_comb(tens):
    t = tens.copy()
    a = check_r2(tl.tensor([t[0],t[1]]))
    b = check_r2(tl.tensor([t[0],t[2]]))
    c = check_r2(tl.tensor([t[1],t[2]]))
    a1 = False
    a2 = False
    a3 = False
    if a:
        a1 = test_r2(t)
        if a1:
            return a1
    if b:
        a2 = test_r2(tl.tensor([t[0],t[2],t[1]]))
        if a2:
            return a2
    if c:
        a3 = test_r2(tl.tensor([t[1],t[2],t[0]]))
        if a3:
            return a3
    return a1 or a2 or a3

In [5]:

# assumes the nonnegative rank 2 subtensor are the first two slices
def test_simple(tens, M, A1, tol = 0.0001):
    a,b = symbols('a,b')
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    a1 = solve(M1.det(),a)[0]
    A1 = A1.subs(a,a1)
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M1_sub = M1[1:4,0:3]
    R = M1[1:4,3]
    res1 = M1_sub.inv() @ R
    res1[0] = fraction(simplify(res1[0]))[0] * fraction(simplify(res1[0]))[1]
    res1[1] = fraction(simplify(res1[1]))[0] * fraction(simplify(res1[1]))[1]
    res1[2] = fraction(simplify(res1[2]))[0] * fraction(simplify(res1[2]))[1]
    a2 = fraction(simplify(a1))[0] * fraction(simplify(a1))[1]
    s1 = solveset(a2 >= 0, b, S.Reals)
    s2 = solveset(res1[0] >= 0, b, S.Reals)
    s3 = solveset(res1[1] >= 0,b,S.Reals)
    s4 = solveset(res1[2] >= 0,b,S.Reals)
    s5 = solveset(b >= 0,b,S.Reals)
    sol = Intersection(s1,s2,s3,s4,s5)
    return sol.measure > tol

# initializes the matrix with the appropriate rank 1 terms given the tensors has nonneg rank 2 subtensor
def init_mat(tens):
    A = Matrix(tens[0]) * Matrix(tens[1]).inv()
    #a1
    res1 = Matrix(A.eigenvects()[0][2])
    #b1
    res2 = Matrix(A.eigenvects()[1][2])

    B = Matrix(tens[0]).transpose() * Matrix(tens[1]).transpose().inv()
    #a2
    res3 = Matrix(B.eigenvects()[0][2])
    #b2
    res4 = Matrix(B.eigenvects()[1][2])

    T1 = np.kron(res1,res3) * 1.0
    T2 = np.kron(res2,res4) * 1.0
    assert(np.all(T1 >= 0) or np.all(T1 <= 0))
    assert(np.all(T2 >= 0) or np.all(T2 <= 0))
    T1 = Matrix(abs(T1))
    T2 = Matrix(abs(T2))

    return Matrix([[T1,T2]])



    
# checks if a given 2x2x3 has nonneg rank 2 subtensor and if it does check if the third slice is a linear combination
def check_simple(tens):
    t = tens.copy()
    c1 = check_r2(tl.tensor([t[0],t[1]]))
    c2 = check_r2(tl.tensor([t[0],t[2]]))
    c3 = check_r2(tl.tensor([t[1],t[2]]))
    a1 = False
    a2 = False
    a3 = False
    if c1:
        a,b = symbols('a,b')
        A1 = Matrix([a*b,a,b,1])
        A2 = Matrix([a,a*b,1,b])
        A3 = Matrix([b,1,a*b,a])
        A4 = Matrix([1,b,a,a*b])
        temp = t
        M = init_mat(temp)
        cond1 = test_simple(temp,M,A1) or test_simple(temp,M,A2) or test_simple(temp,M,A3) or test_simple(temp,M,A4)
        if cond1:
            return cond1
    if c2:
        a,b = symbols('a,b')
        A1 = Matrix([a*b,a,b,1])
        A2 = Matrix([a,a*b,1,b])
        A3 = Matrix([b,1,a*b,a])
        A4 = Matrix([1,b,a,a*b])
        temp = tl.tensor([t[0],t[2],t[1]])
        M = init_mat(temp)
        cond2 = test_simple(temp,M,A1) or test_simple(temp,M,A2) or test_simple(temp,M,A3) or test_simple(temp,M,A4)
        if cond2:
            return cond2
    if c3:
        a,b = symbols('a,b')
        A1 = Matrix([a*b,a,b,1])
        A2 = Matrix([a,a*b,1,b])
        A3 = Matrix([b,1,a*b,a])
        A4 = Matrix([1,b,a,a*b])
        temp = tl.tensor([t[1],t[2],t[0]])
        M = init_mat(temp)
        cond3 = test_simple(temp,M,A1) or test_simple(temp,M,A2) or test_simple(temp,M,A3) or test_simple(temp,M,A4)
        if cond3:
            return cond3
    return False

def loop_rotations(i):
    tens = rand_tensor()
    tens = rank_tree(size = 50)
    tens = tensors[i]
    ret = [0] *11
    if proc_tensor(tens) or proc_tensor(mat_trans(tens)) or proc_tensor(mat_inv(tens)) or proc_tensor(rotate(tens)):
        ret[1] = 1
        ret[0] = 1
    if check_simple(tens):
        ret[2] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 2)
    r4 = check_r4(tens)
    if r4:
        ret[4] = 1
    if r4 and r2:
        ret[3] = 1
    if not r4 and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    tensors[i] = (tensors[i],ret[0])
    return ret

def loop_rotations_old(i):
    tens = rand_tensor()
    tens = rank_tree(size = 50)
    tens = tensors[i]
    ret = [0] *11
    if proc_tensor(tens) or proc_tensor(mat_trans(tens)) or proc_tensor(mat_inv(tens)) or proc_tensor(rotate(tens)):
        ret[1] = 1
        ret[0] = 1
    if check_comb(tens):
        ret[2] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 2)
    r4 = check_r4(tens)
    if r4:
        ret[4] = 1
    if r4 and r2:
        ret[3] = 1
    if not r4 and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    return ret

In [6]:
tensors = []
for i in range(10000):
    tensors.append(rand_tensor())

In [301]:
te = time.time()
total = 1000
results = Parallel(n_jobs=6)(delayed(loop_rotations_old)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[894, 790, 382, 0, 0, 0, 534, 58, 104, 0, 0]
128.48300099372864


In [15]:
te = time.time()
total = 1000
results = Parallel(n_jobs=5)(delayed(loop_rotations)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[923, 760, 600, 0, 0, 0, 553, 50, 163, 0, 0]
959.0324437618256


In [307]:
te = time.time()
total = 1000
results = Parallel(n_jobs=6)(delayed(loop_rotations_old)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[156, 132, 36, 0, 499, 0, 15, 2, 24, 0, 0]
127.79600024223328


In [310]:
te = time.time()
total = 1000
results = Parallel(n_jobs=6)(delayed(loop_rotations)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[217, 132, 124, 0, 499, 0, 15, 0, 85, 0, 0]
139.09999990463257


In [262]:
c1 = False
c2 = False
c3 = False
tens = rand_tensor()
while not (c2 ):
    tens = rank_tree(size = 60)
    sols = test(tens)
    c1 = sols[7] == 1
    c2 = check_r2(tl.tensor([tens[0],tens[1]]))
print(c1,c2,test(tens))
tens

False True [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]


array([[[45434., 36866.],
        [76844., 64056.]],

       [[43446., 32472.],
        [63736., 48102.]],

       [[37214., 38618.],
        [39024., 38688.]]])

In [58]:
tens = rank_tree(size = 50)
tens

array([[[38366., 38358.],
        [44180., 46668.]],

       [[34083., 63729.],
        [43281., 81855.]],

       [[12140.,  9072.],
        [13196., 10388.]],

       [[26072., 13284.],
        [28310., 14314.]]])

In [23]:
test(tl.tensor([tens[0],tens[1],tens[2]]))

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
wrong = 0
for i in range(1000):
    tens = rank_tree(size = 50)
    res = test(tl.tensor([tens[0],tens[1],tens[2]]))
    if res[0] != 1:
        wrong += 1
print(wrong)

In [78]:
tens = rank_tree(size = 50)
t1 = Matrix(tens[0]).reshape(4,1)
t2 = Matrix(tens[1]).reshape(4,1)
t3 = Matrix(tens[2]).reshape(4,1)
t4 = Matrix(tens[3]).reshape(4,1)
M = Matrix([[t1,t2,t3,t4]])
print(M.det())
M_sub = M[0:3,0:3]
R = M[0:3,3]
sols = M_sub.inv() @ R
print(sols[0] * sols[1] * sols[2])
sols

0
-0.0561799849024696


Matrix([
[  1.02472559214327],
[-0.163200462160603],
[ 0.335932986712884]])

In [87]:
wrong = 0
for i in range(1000):
    tens = rank_tree(size = 50)
    t1 = Matrix(tens[0]).reshape(4,1)
    t2 = Matrix(tens[1]).reshape(4,1)
    t3 = Matrix(tens[2]).reshape(4,1)
    t4 = Matrix(tens[3]).reshape(4,1)
    M = Matrix([[t1,t2,t3,t4]])
    M_sub = M[0:3,0:3]
    R = M[0:3,3]
    sols = M_sub.inv() @ R
    prod = sols[0] * sols[1] * sols[2]
    if prod <=0:
        wrong += 1
        if prod == 0:
            print("flag")
        
print(wrong)

628


In [57]:
test(tl.tensor([tens[0],tens[1],tens[2]]))

[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]